In [1]:
%AddDeps org.apache.spark spark-sql-kafka-0-10_2.11 2.4.5 --transitive

Marking org.apache.spark:spark-sql-kafka-0-10_2.11:2.4.5 for download
Obtained 12 files


Download the connector. Note the _--transitive_ flag- it is necessary in order to download Kafka utils etc

In [2]:
import org.apache.spark._

## Spark Context

In [3]:
val spark = SparkSession.builder().appName("MessageProcessor")
            .master("spark://spark:7077").getOrCreate()

spark = org.apache.spark.sql.SparkSession@2835b9a8


org.apache.spark.sql.SparkSession@2835b9a8

## Obtain dataframes from Kafka

In [4]:
val dataframe = spark
    .readStream
    .format("kafka")
    .option("kafka.bootstrap.servers", "kafka1:29092")
    .option("subscribe", "Messages")
    .option("startingOffsets", "earliest")
    .option("endingOffsets", "latest")
    .load()

dataframe = [key: binary, value: binary ... 5 more fields]


[key: binary, value: binary ... 5 more fields]

In [6]:
dataframe.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")
  .as[(String, String)]

[key: string, value: string]

In [7]:
dataframe.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [11]:
val output = dataframe.writeStream
                    .outputMode("append").format("memory")
                    .queryName("table")
                    .start()

output = org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@79b4a606


org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@79b4a606

In [12]:
while (output.isActive) {
    Thread.sleep(10000)
    spark.sql("select * from table").show()
}

+---+-----+-----+---------+------+---------+-------------+
|key|value|topic|partition|offset|timestamp|timestampType|
+---+-----+-----+---------+------+---------+-------------+
+---+-----+-----+---------+------+---------+-------------+

+--------------------+--------------------+--------+---------+------+--------------------+-------------+
|                 key|               value|   topic|partition|offset|           timestamp|timestampType|
+--------------------+--------------------+--------+---------+------+--------------------+-------------+
|[52 6F 77 44 61 7...|[7B 22 6B 6F 6C 3...|Messages|        0|     3|2020-05-19 17:15:...|            0|
|[52 6F 77 44 61 7...|[7B 22 6B 6F 6C 3...|Messages|        0|     4|2020-05-19 17:15:...|            0|
|[52 6F 77 44 61 7...|[7B 22 6B 6F 6C 3...|Messages|        0|     5|2020-05-19 17:15:...|            0|
+--------------------+--------------------+--------+---------+------+--------------------+-------------+

+--------------------+-----

Name: java.lang.InterruptedException
Message: sleep interrupted
StackTrace:   at java.lang.Thread.sleep(Native Method)

In [10]:
output.stop()

lastException: Throwable = null
